In [1]:
from src.utils import * 
import pandas as pd

base_dir = get_dir_name(True)
try:
    os.mkdir(base_dir + "/feature_set")
except FileExistsError:
    pass
sym = 0
# together_csv(base_dir + "/train_set", base_dir + f"/feature_set/no_feature_sym{sym}.csv", sym)
no_feature_sym0 = pd.read_csv(base_dir + f"/feature_set/no_feature_sym{sym}.csv")

In [2]:
import numpy as np
from sklearn.model_selection import train_test_split 
from src.feature_deal import *
import joblib

In [7]:
%reload_ext autoreload  
%autoreload 2    
df = no_feature_sym0.copy()
y = df["label_5"]
feature_col_names = main_feature(df, True)
df = main_feature(df)
X = df[feature_col_names]

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)


In [9]:
from lightgbm import LGBMClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, recall_score
import pandas as pd


def gridsearch():
    param_grid = {
        "learning_rate": [0.01, 0.05, 0.1,],
        "max_depth":     [3, 5, 7],
        "subsample":     [0.8, 1.0],
        "n_estimators":  [100, 200, 300]
    }

    grid_search = GridSearchCV(
        estimator=LGBMClassifier(random_state=42, objective='binary'),  # 如果是二分类
        param_grid=param_grid,
        cv=3,
        scoring='accuracy',   
        n_jobs=10
    )


    grid_search.fit(X_train, y_train)
    best_params = grid_search.best_params_
    print("最佳参数：", best_params)
    return best_params
best_params = gridsearch()
def train(best_params):

    lgb_model = LGBMClassifier(**best_params, random_state=42, objective='multiclass', n_jobs=4)
    lgb_model.fit(X_train, y_train)

    y_pred = lgb_model.predict(X_val)

    acc = accuracy_score(y_val, y_pred)
    rec = recall_score(y_val, y_pred, average='macro')    

    print(f"验证集准确率 (Accuracy): {acc:.4f}")
    print(f"验证集召回率 (Recall):   {rec:.4f}")
# best_params = {'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 300, 'subsample': 1}
train(best_params)

最佳参数： {'learning_rate': 0.1, 'max_depth': 7, 'n_estimators': 300, 'subsample': 0.8}
验证集准确率 (Accuracy): 0.6607
验证集召回率 (Recall):   0.3843


In [5]:
# joblib.dump(lgb_model, base_dir + f"/models/lgb_sym{sym}_acc{acc:.3f}.pkl")

In [6]:
one_pred_model = joblib.load(base_dir + f"/models/lgb_sym{sym}_acc0.657.pkl")
# y_pred_all = lgb_model.predict(X)
# y_pred_one = one_pred_model.predict(X)
# X["y_pred_all"] = y_pred_all
# X["y_pred_one"] = y_pred_one
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)
train(best_params)

验证集准确率 (Accuracy): 0.6590
验证集召回率 (Recall):   0.3833
